# Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 3. Обработка текстов.

### Общая информация

Дата выдачи: 13.01.2022

Мягкий дедлайн: 23:59MSK 6.02.2022

Жесткий дедлайн: 23:59MSK 10.02.2022

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{penalty} = M_{full} \cdot 0.85^{t/1440}$, где $M_{full}$ — полная оценка за работу без учета штрафа, а $t$ — время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше 8.5, а если сдать перед самым жестким дедлайном, то ваш максимум — 5.22 балла.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В данном домашнем задании вам предстоит предсказывать пользовательскую оценку отеля по тексту отзыва. Нужно обучиться на данных с кэггла и заслать в [соревнование](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) предикт. По той же ссылке можете скачать данные.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву.

Главная метрика - Mean Absolute Error (MAE). Во всех частях домашней работы вам нужно получить значение MAE не превышающее 0.92 на публичном лидерборде. В противном случае мы будем вынуждены не засчитать задание :( 

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Для локального тестирования используйте предоставленное разбиение на трейн и тест.

Good luck & have fun! 💪

#### Использовать любые данные для обучения кроме предоставленных организаторами строго запрещено. В последней части можно использовать предобученные модели из библиотеки `transformers`.

In [1]:
PATH_TO_TRAIN_DATA = 'data/train.csv'

In [10]:
import pandas as pd

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7


Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [11]:
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def process_text(text):
    # remove empty parts
    text = text.replace('No Negative', '').replace('No Positive', '')
    # tokenize
    words = word_tokenize(text)
    # remove all punctuation
    words = [word.strip(string.punctuation) for word in words]
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # stem words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # remove small words
    words = [word for word in words if len(word) > 1]
    return words

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from pandarallel import pandarallel
pandarallel.initialize()

df['negative'] = df['negative'].parallel_apply(process_text)
df['positive'] = df['positive'].parallel_apply(process_text)
df['review'] = df['positive'] + df['negative']

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [25]:
df.to_pickle('data/processed_ipynb.csv')

In [27]:
df2 = pd.read_pickle('data/processed_ipynb.csv')

In [28]:
df.head()

,review_id,negative,positive,score,review
0,00003c6036f30f590c0ac435efb8739b,"[there, issu, wifi, connect]",[],7.1,"[there, issu, wifi, connect]"
1,00004d18f186bf2489590dc415876f73,"[tv, work]",[],7.5,"[tv, work]"
2,0000cf900cbb8667fad33a717e9b1cf4,"[more, pillow]","[beauti, room, great, locat, love, staff]",10.0,"[beauti, room, great, locat, love, staff, more..."
3,0000df16edf19e7ad9dd8c5cd6f6925e,"[veri, busi]",[locat],5.4,"[locat, veri, busi]"
4,00025e1aa3ac32edb496db49e76bbd00,"[room, could, bit, refurbish, could, corridor,...","[nice, breakfast, handi, victoria, train, stat...",6.7,"[nice, breakfast, handi, victoria, train, stat..."


In [29]:
df2.head()

,review_id,negative,positive,score,review
0,00003c6036f30f590c0ac435efb8739b,"[there, issu, wifi, connect]",[],7.1,"[there, issu, wifi, connect]"
1,00004d18f186bf2489590dc415876f73,"[tv, work]",[],7.5,"[tv, work]"
2,0000cf900cbb8667fad33a717e9b1cf4,"[more, pillow]","[beauti, room, great, locat, love, staff]",10.0,"[beauti, room, great, locat, love, staff, more..."
3,0000df16edf19e7ad9dd8c5cd6f6925e,"[veri, busi]",[locat],5.4,"[locat, veri, busi]"
4,00025e1aa3ac32edb496db49e76bbd00,"[room, could, bit, refurbish, could, corridor,...","[nice, breakfast, handi, victoria, train, stat...",6.7,"[nice, breakfast, handi, victoria, train, stat..."


In [31]:
from sklearn.model_selection import train_test_split

X = df2.drop(columns=['review_id','score'])
y = df2.score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1412) # <- для локального тестирования

In [32]:
X_train

,negative,positive,review
41766,"[room, tini, price, exhorbit, size, room]",[],"[room, tini, price, exhorbit, size, room]"
99660,[],"[excel, staff, hotel]","[excel, staff, hotel]"
19507,"[veri, facil, eg, coffe, shop, leisur, facil, ...","[the, staff, went, way, friendli, welcom]","[the, staff, went, way, friendli, welcom, veri..."
72103,"[the, receptionist, desk, need, improv, lack, ...","[the, locat, superb, we, night, pari, and, mak...","[the, locat, superb, we, night, pari, and, mak..."
69073,"[park, option, well, solv]","[perfect, locat, valu, money]","[perfect, locat, valu, money, park, option, we..."
...,...,...,...
33144,"[extrem, imperson, overrun, peopl, mill, around]",[],"[extrem, imperson, overrun, peopl, mill, around]"
65200,"[couldn, find, dislik]","[veri, profession, staff, throughout, hotel, e...","[veri, profession, staff, throughout, hotel, e..."
49810,[],[],[]
86805,"[bed, bit, soft, fell, one, point, lot, charge...","[locat, staff, rooftop, pool]","[locat, staff, rooftop, pool, bed, bit, soft, ..."


In [6]:
y_train.shape

(75000,)

### Часть 1. 1 балл

In [14]:
X_train.to_csv('X1')

Обучите логистическую или линейную регрессию на TF-IDF векторах текстов.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, make_scorer

pipeline = Pipeline([
    ('tfidf', ColumnTransformer([
        ('tfidf_p', TfidfVectorizer(preprocessor=lambda x: ' '.join(x)), 'positive'),
        ('tfidf_n', TfidfVectorizer(preprocessor=lambda x: ' '.join(x)), 'negative'),
        ('tfidf', TfidfVectorizer(preprocessor=lambda x: ' '.join(x)), 'review'),
    ])),
    ('regression', Ridge())
])

clf = pipeline.fit(X_train, y_train)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [13]:
print(f'Train MAE: {mean_absolute_error(y_train, clf.predict(X_train))}')
print(f'Test MAE: {mean_absolute_error(y_test, clf.predict(X_test))}')

Train MAE: 0.6868365424851248
Test MAE: 0.8301496175958935


Предскажите этой моделью тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

### Часть 2. 2 балла

Обучите логистическую или линейную регрессию на усредненных Word2Vec векторах. 

Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (Inverse document frequency)

In [ ]:
def calc_idf(texts):
    pass

Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга. 

#### Сделайте выводы:

Теперь попробуйте обучить логистическую или линейную регрессию на любых других эмбеддингах размерности 300 и сравните качество с Word2Vec.
#### Выводы:
`<ВАШ ТЕКСТ ЗДЕСЬ>`

Предскажите вашей лучшей моделью из этого задания тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

### Часть 3. 4 балла

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
WORDS = set()
for sent in list(df['positive']):
    for w in sent:
        WORDS.add(w)
        
for sent in list(df['negative']):
    for w in sent:
        WORDS.add(w)

In [ ]:
int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: ii for ii, w in int2word.items()}

In [ ]:
MAX_LEN = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))

In [ ]:
MAX_LEN

399

In [ ]:
from torch.nn.utils.rnn import pad_sequence

train_pos_pad = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in df_train['positive']], 
                           batch_first=True)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        pass
        
    def __len__(self):
        ## TODO
        pass
    
    def __getitem__(self, idx):
        ## TODO
        pass

In [ ]:
BATCH_SIZE = 1

train_dataset = ReviewsDataset(df_train)
test_dataset = ReviewsDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
NUM_EPOCHS = 1

for n in range(NUM_EPOCHS):
    model.train()
    ## TODO

### Контест (до 3 баллов)

По итогам всех ваших экспериментов выберите модель, которую считаете лучшей. Сделайте сабмит в контест. В зависимости от вашего скора на публичном лидерборде, мы начислим вам баллы:

 - <0.76 - 3 балла
 - [0.76; 0.78) - 2 балла
 - [0.78; 0.8) - 1 балл